In [0]:
%pylab inline

In [0]:
import dataiku
from dataiku import pandasutils as pdu
import pandas as pd
import lasio
import os.path
from func import LasReader
from newdeg import NewDeg
from newlat import NewLat


handle = dataiku.Folder("My_folder")
path = handle.get_path()

with open(os.path.join(path, "LAS.las")) as f:
    las = LasReader(f)
    df = pd.DataFrame(las.well_info, index=[0,])
    for col in df.columns:
        if 'NULL' in col:
            del df[col]
        if ' ' in col:
            del df[col]
result = df.replace('-999.25', '')
# Создаю регулярные выражения для поиска координат
reg_expr = r'(\d+�\s\d+.\s\d+.\d+.\s[N])'
reg_expr2 = r'(\d+�\s\d+.\s\d+.\d+.\s[E])'
reg_expr3 = r'(^\d+.\d+)'
reg_expr4 = r'(\?+)'

long_lst = list(result['LONG'])
lati_lst = list(result['LATI'])
long_lst2 = long_lst.copy()
lati_lst2 = lati_lst.copy()

new_deg =  NewDeg(reg_expr3, long_lst2)
new_deg2 = NewDeg(reg_expr3, lati_lst2)

new_lati = NewLat(reg_expr, new_deg2.lst)
new_long = NewLat(reg_expr2, new_deg.lst)

result['LATI']= result['LATI'].replace(lati_lst,new_lati.lst)
result['LONG'] = result['LONG'].replace(long_lst, new_long.lst)

final_data = result.dropna(thresh=result.shape[0] //2, axis=1)
final_data=final_data.replace(to_replace=reg_expr4, value='mm', regex=True)

    
dataset_name = 'test5'

# Get a handle to the current project
client = dataiku.api_client()
project = client.get_project(dataiku.default_project_key())

# Create a SQL dataset (you can create other types by specifying different parameters for the with_store_into method)
# Documentation here: https://doc.dataiku.com/dss/latest/python-api/datasets-other.html#programmatic-creation-and-setup-managed-datasets
# Note that documentation shows project.new_managed_dataset which is incorrect
builder = project.new_managed_dataset_creation_helper(dataset_name)
builder.with_store_into("las")
builder.create() 

# Write dataframe to dataset
dataiku.Dataset(dataset_name).write_with_schema(final_data)
    



